<a href="https://colab.research.google.com/github/tallywiesenberg/DS-Unit-2-Regression-Classification/blob/master/assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [ ] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [ ] Engineer at least two new features. (See below for explanation & ideas.)
- [ ] Fit a linear regression model with at least two features.
- [ ] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [ ] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Do the [Plotly Dash](https://dash.plot.ly/) Tutorial, Parts 1 & 2.
- [ ] Add your own stretch goal(s) !

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [0]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module1')

In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [87]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv('../data/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
##ENGINEER NEW FEATURE: ONE-HOT ATTRIBUTE SCORE
df['one_hot_score'] = [df.iloc[index, 10:].mean() for index in range(len(df.index))]

In [0]:
##ENGINEER NEW FEATURE: NUMBER OF ROOMS
df['number_of_rooms'] = df['bathrooms'] + df['bedrooms']

In [98]:
df['number_of_rooms'].head()

0    4.5
1    3.0
2    2.0
3    2.0
4    5.0
Name: number_of_rooms, dtype: float64

In [0]:
##SPLIT TEST AND TRAIN DATA

#define features and target
features = ['latitude', 'longitude', 'bedrooms', 'bathrooms', 'one_hot_score']
target = 'price'

X = df[features]
y = df[target]

#split train and test by months: April and May are Train, June is Test
mask_train = (df['created'] >= '2016-04-01') & (df['created'] <= '2016-05-31')
X_train = X.loc[mask_train]

X_test = X.loc[mask_test]

mask_test = (df['created'] >= '2016-06-01') & (df['created'] <= '2016-06-30')
y_train = y.loc[mask_train]

y_test = y.loc[mask_test]

In [0]:
##FIT LINEAR REGRESSION

#load model
model = LinearRegression()

#fit model
model.fit(X_train, y_train)

#predict values
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [101]:
coefficients = model.coef_[:]
print('Model Feature Coefficients:', coefficients)
intercept = model.intercept_
print('Model Intercept:', intercept)

Model Feature Coefficients: [  1825.13298585 -14912.21035826    426.77674552   1927.33841497
   1116.35995772]
Model Intercept: -1177090.1078350258


In [103]:
#REGRESSION METRICS

#Metric 1: Root Mean Squared Error
mean_squared = mean_squared_error(y_train, y_pred_train)
rmse_train = np.sqrt(mean_squared)
print('Root Mean Squared Error (aka the mean of the errors squared) of the Training set:', rmse_train)

mean_squared = mean_squared_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared)
print('Root Mean Squared Error (aka the mean of the errors squared) of the Test set:', rmse_test)

#Metric 2: Mean Absolute Error
MAE = mean_absolute_error(y_train, y_pred_train)
print('Mean Absolute Error of the Training Set:', MAE )

MAE = mean_absolute_error(y_test, y_pred_test)
print('Mean Absolute Error of the Test Set:', MAE)

#Metric 3: R^2
r2 = r2_score(y_train, y_pred_train)
print('R^2 Score of the Training Set:', r2)

r2 = r2_score(y_test, y_pred_test)
print('R^2 Score of the Test Set:', r2)

Root Mean Squared Error (aka the mean of the errors squared) of the Training set: 1136.3217931285678
Root Mean Squared Error (aka the mean of the errors squared) of the Test set: 1121.7902507979268
Mean Absolute Error of the Training Set: 729.203048933191
Mean Absolute Error of the Test Set: 733.7193076265431
R^2 Score of the Training Set: 0.5841951308690403
R^2 Score of the Test Set: 0.5951078649938724
